In [2]:
import pandas as pd
import numpy as np
import wrds

In [3]:
# Establish connection with wrds
conn = wrds.Connection()

Enter your WRDS username [komalniraula]: kn2505
Enter your password: ········


WRDS recommends setting up a .pgpass file.


Create .pgpass file now [y/n]?:  n


You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


In [4]:
# Data call
years = list(range(2014, 2025))
df_list = []

for i, year in enumerate(years):
    print(f"Loading {year} ({i+1}/{len(years)})...")

    start_date = f"{year}-01-01"
    end_date = f"{year}-12-31"

    query = f"""
        SELECT 
            date,
            dff,
            effr
        FROM frb.rates_daily
        WHERE date BETWEEN '{start_date}' AND '{end_date}'
              AND (dff IS NOT NULL OR effr IS NOT NULL)
        ORDER BY date
    """

    df_year = conn.raw_sql(query, date_cols=['date'])
    df_list.append(df_year)

df_fedfunds = pd.concat(df_list, ignore_index=True)
df_fedfunds

Loading 2014 (1/11)...
Loading 2015 (2/11)...
Loading 2016 (3/11)...
Loading 2017 (4/11)...
Loading 2018 (5/11)...
Loading 2019 (6/11)...
Loading 2020 (7/11)...
Loading 2021 (8/11)...
Loading 2022 (9/11)...
Loading 2023 (10/11)...
Loading 2024 (11/11)...


,date,dff,effr
0,2014-01-01,0.07,<NA>
1,2014-01-02,0.08,0.08
2,2014-01-03,0.08,0.08
3,2014-01-04,0.08,<NA>
4,2014-01-05,0.08,<NA>
...,...,...,...
4013,2024-12-27,4.33,4.33
4014,2024-12-28,4.33,<NA>
4015,2024-12-29,4.33,<NA>
4016,2024-12-30,4.33,4.33


- **effr** is the Effective Federal Funds Rate which is daily calculated from actual overnight transactions between depository institutions. 
- **dff** is Federal Funds Effective Rate published by the St. Louis Fed. 

We will primarily use effr and if there is no effr value then only consider dff

In [6]:
df_fedfunds['fed_funds_rate'] = df_fedfunds['effr'].fillna(df_fedfunds['dff'])

df_fedfunds

,date,dff,effr,fed_funds_rate
0,2014-01-01,0.07,<NA>,0.07
1,2014-01-02,0.08,0.08,0.08
2,2014-01-03,0.08,0.08,0.08
3,2014-01-04,0.08,<NA>,0.08
4,2014-01-05,0.08,<NA>,0.08
...,...,...,...,...
4013,2024-12-27,4.33,4.33,4.33
4014,2024-12-28,4.33,<NA>,4.33
4015,2024-12-29,4.33,<NA>,4.33
4016,2024-12-30,4.33,4.33,4.33


In [7]:
df_fedfunds[['date', 'fed_funds_rate']].to_csv('fed_rates.csv', index = False) # save